# TensorFlow Deep Neural Net

Here I will be re-implementing a deep neural net that I put together in a previous notebook, this time using the TensorFlow low-level API to gain familiarity and perhaps achieve better performance (both speed and accuracy). 

In [1]:
import tensorflow as tf

I will be using [this](https://www.tensorflow.org/get_started/mnist/pros) to help me get started.  I also used a script found [here](https://gist.githubusercontent.com/ischlag/41d15424e7989b936c1609b53edd1390/raw/5ed7aca47bcca30b3df1c3bfd0f027e6bcdb430c/mnist-to-jpg.py) to pre-process the MNIST data to more closely resemble something I could put together from a random dataset.

In [2]:
# Tensors for input layer and output layer
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
BATCH_SIZE = 25

In [33]:
from tensorflow.contrib.data import Dataset, Iterator

dataset_path      = "mnist/"
test_labels_file  = "test-labels.csv"
train_labels_file = "train-labels.csv"

def read_label_file(file):
    f = open(file, "r")
    filepaths = []
    labels = []
    for line in f:
        filepath, label = line.split(",")
        filepaths.append(filepath)
        labels.append(int(label))
    return filepaths, labels

test_filepaths, test_labels = read_label_file(dataset_path + test_labels_file)
train_filepaths, train_labels = read_label_file(dataset_path + train_labels_file)

NUM_CLASSES = 10

def input_parser(img_path, label):
    # convert the label to one-hot encoding
    one_hot = tf.one_hot(label, NUM_CLASSES)

    # read the img from file
    img_file = tf.read_file(img_path)
    img_decoded = tf.image.decode_image(img_file)

    return img_decoded, one_hot

# Create Dataset objects
train_data = Dataset.from_tensor_slices((train_filepaths, train_labels))
test_data = Dataset.from_tensor_slices((test_filepaths, test_labels))

train_data = train_data.map(input_parser, num_threads=8, output_buffer_size=100*BATCH_SIZE)
test_data = test_data.map(input_parser, num_threads=8, output_buffer_size=100*BATCH_SIZE)

# Create Iterator objects
iterator = Iterator.from_structure(train_data.output_types,
                                   train_data.output_shapes)
next_element = iterator.get_next()

# Create two initialization ops to switch between the datasets
training_init_op = iterator.make_initializer(train_data)
testing_init_op = iterator.make_initializer(test_data)

So now I have two ops, each of which will allow me to iterate over the Datasets I created.  One corresponds to training data and the other to testing/evaluation data.  Below I print the shape of the first element of each to see that the processing has worked as expected.

In [35]:
with tf.Session() as sess:

    sess.run(training_init_op)

    try:
        elem = sess.run(next_element)
        print(str(tf.shape(elem[0])) + ", " + str(tf.shape(elem[1])))
    except tf.errors.OutOfRangeError:
        print("End of training dataset.")

    sess.run(testing_init_op)

    try:
        elem = sess.run(next_element)
        print(str(tf.shape(elem[0])) + ", " + str(tf.shape(elem[1])))
    except tf.errors.OutOfRangeError:
        print("End of test dataset.")

Tensor("Shape_29:0", shape=(3,), dtype=int32), Tensor("Shape_30:0", shape=(1,), dtype=int32)
Tensor("Shape_31:0", shape=(3,), dtype=int32), Tensor("Shape_32:0", shape=(1,), dtype=int32)


The first element in each tuple is a 28x28x1 array, with the third dimension corresponding to color channels.  The second element in each tuple is an array of NUM_CLASSES integers and is one-hot.

In [43]:
HIDDEN_LAYERS = [30]
WEIGHTS = []
BIASES = []

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

for layer in HIDDEN_LAYERS:
    WEIGHTS.append(weight_variable([layer]))
    BIASES.append(bias_variable([layer]))